<h2>
Thoughts
</h2>

<p>
Every comment in the dataset identified by the "in_reply_to..." field must be linked to its corresponding post. The issue that arises is that it's first necessary to load the entire dataset and search through the entire thing for ever single comment. My first thought is to utilize a hashmap in order to attain O(1) complexity for the problem.  
</p>

In [37]:
import numpy as np
import pandas as pd
from glob import glob
import math
from IPython.display import clear_output

In [2]:
filenames = glob("../data/BTC/*.csv")

# takes about 1.5 min on my machine to load the entire dataset
df = pd.concat([pd.read_csv(filename) for filename in filenames])

# drop null username tweets
df["user_name"].dropna(inplace=True)
df["in_reply_to_status_id"].dropna(inplace=True)
df["in_reply_to_screen_name"].dropna(inplace=True)
df["created_at"].dropna(inplace=True)

/tmp/ipykernel_9811/2313426320.py:4: DtypeWarning: Columns (0,9) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(filename) for filename in filenames])
/tmp/ipykernel_9811/2313426320.py:4: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(filename) for filename in filenames])


In [8]:

# create a python hashmap for the user_name column so we can access it O(1)
# from the in_reply_to_screen_name attribute

user_comments_dict = {}

# create dictionary which maps usernames to a dictionary of 
for index, user_name in enumerate(df["in_reply_to_screen_name"]): 

    # skip nan values
    if type(user_name) == float:
        continue

    if not(user_name in user_comments_dict):
        user_comments_dict[user_name] = set([index])
    else:
        user_comments_dict[user_name].add(index)

In [80]:
# Order the results to verify integrity (should see celebrities and influencers at top of list /w thousands of comments)
# ordered = sorted([(key, len(val)) for key, val in user_name_dict.items()], key=lambda e: e[1], reverse=True)
# ordered
# df[df["user_name"] == "Bitcoin"]

In [56]:
comments_series = pd.Series(index=user_comments_dict.keys(), data=user_comments_dict.values())

commented_posts_df = df.merge(comments_series.rename("comments"), how='inner', left_on=df['user_name'], right_index=True)

In [82]:
commented_posts_df[commented_posts_df["user_name"] == "Bitcoin"]

,key_0,coordinates,created_at,hashtags,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_statuses_count,user_urls,user_verified,comments
60,Bitcoin,NaN,Fri Sep 06 22:38:09 +0000 2019,NaN,NaN,0,1.170104e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
103,Bitcoin,NaN,Fri Sep 06 22:40:09 +0000 2019,NaN,NaN,0,1.170104e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
302,Bitcoin,NaN,Fri Sep 06 22:46:50 +0000 2019,NaN,NaN,0,1.170106e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
371,Bitcoin,NaN,Fri Sep 06 22:50:00 +0000 2019,NaN,NaN,0,1.170107e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
878,Bitcoin,NaN,Fri Sep 06 23:07:15 +0000 2019,NaN,NaN,0,1.170111e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997729,Bitcoin,NaN,Fri Sep 06 21:09:19 +0000 2019,NaN,NaN,0,1.170082e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
1998486,Bitcoin,NaN,Fri Sep 06 21:36:49 +0000 2019,NaN,NaN,0,1.170088e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
1998919,Bitcoin,NaN,Fri Sep 06 21:56:20 +0000 2019,NaN,NaN,0,1.170093e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."
1999340,Bitcoin,NaN,Fri Sep 06 22:08:33 +0000 2019,NaN,NaN,0,1.170096e+18,NaN,NaN,NaN,...,2,216,2,5,"Texas, USA",Bitcoin,106764,https://zabo.com,False,"{4096000, 1966088, 1179657, 3997714, 786465, 3..."


In [38]:


# loop through comments
# create a link for user -> user_name_dict[user.user_name] -> user_name_dict[user.user_name][]

# create new series for the comments
comments_series = pd.Series(index=df.index, dtype=object)

for row_index, row in enumerate(df.itertuples(True)):

    user_name = row.user_name
    
    # if this user has comments
    if user_name in user_comments_dict:
        comments_series[row_index] = user_comments_dict[user_name]

    if row_index % 100000 == 0:
        clear_output()
        print(f"processing {row_index/len(comments_series):.2f}%", end="")
        # print(f"processing {row_index}/{len(comments_series)}")




processing 0.05

KeyboardInterrupt: 

In [31]:
comments_series[comments_series.notna()].index

Int64Index([30, 60, 79, 81, 92, 30, 60, 79, 81, 92], dtype='int64')